In [1]:
import json
import torch
from nmr2struct.spec2frags_model import Transformer, generate, Spec2Gen
from tqdm import tqdm

In [2]:
specs_vocab_size = 300
# Correct
frags_vocab_size = 5113

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
model = Transformer(
    src_vocab_size=specs_vocab_size + 3,
    tgt_vocab_size=frags_vocab_size + 3,
    d_model=256,
    num_heads=8,
    num_layers=4,
    d_ff=256,
    max_seq_length=100,
    dropout=0.2
)

checkpoint = 'nmr2struct_up_propper_fixed_frags_vocab_10'

model.load_state_dict(torch.load(f'nmr2struct/checkpoints/{checkpoint}.pt', map_location = device))

/tmp/ipykernel_3563112/445455067.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'nmr2struct/checkpoints/{checkpoint}.pt', map_locatio

<All keys matched successfully>

In [4]:
with open('nmr2struct/fp2frags.json') as f:
    fp2_dict = json.loads(f.read())

fp2 = []
for x in fp2_dict.keys():
    fp2.append(x)

In [5]:
input_spectra = [
    [149.1, 140.6, 127.0, 126.7, 124.0, 119.8, 42.4, 18.2],
    [161.8, 135.18, 134.5, 130.25, 129.82, 127.9, 127.11]
                ]

In [6]:
signals = Spec2Gen(data=input_spectra)

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 906.09it/s]


In [7]:
gen_frags = []
num_gen = 50

for signal in signals:
    frag_sm_list = []
    for _ in range(num_gen):
        frag_sm = []
        
        frags = generate(signal, model)
        
        for x in set(frags):
            if x < len(fp2):
                frag_sm.append(fp2[x])
            else:
                frag_sm.append('')
        frag_sm_list.append(frag_sm)
    gen_frags.append(frag_sm_list)